In [1]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileId = drive.CreateFile({'id': '1fbvd-FVNUvU8xf6hpwYzpBwENpQTPVfl'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

!unzip temp.zip -d ./

imdb_master.zip
Archive:  temp.zip
replace ./imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./imdb_master.csv       


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [0]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
def clean_reviews(text):
    lemmatizer = WordNetLemmatizer()
    my_stopwords = stopwords.words('english') 
    text = text.replace("<br >", "")
    text = text.replace("</br >", "")        
    text = re.sub('[^a-zA-Z]',' ', text)
    text = text.lower() 
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in my_stopwords]
    text = " ".join(text)
    return text  

In [0]:
df["review"]=df.review.apply(lambda x: clean_reviews(x))

In [7]:
df.head()

,review,label
0,first hardly ever watch swedish movie actua...,pos
1,somehow movie manage invigorate bittersweet ...,pos
2,coach use good ha move new area get away tea...,neg
3,good rent buy original watch someone ha gun ...,neg
4,great horror comedy michael davis iwas laugh h...,pos


In [8]:
df["label"] = df["label"].map({'pos': 1, 'neg': 0})
df.head()

,review,label
0,first hardly ever watch swedish movie actua...,1
1,somehow movie manage invigorate bittersweet ...,1
2,coach use good ha move new area get away tea...,0
3,good rent buy original watch someone ha gun ...,0
4,great horror comedy michael davis iwas laugh h...,1


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["label"], test_size=0.2)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional
from keras.layers.embeddings import Embedding
import string

Using TensorFlow backend.


In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
data = pad_sequences(sequences, maxlen=200)


In [12]:
model = Sequential()
model.add(Embedding(vocabulary_size, 128, input_length=200))
model.add(LSTM(500, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(data, df['label'], validation_split=0.2, epochs=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 1098s 27ms/step - loss: 0.4247 - acc: 0.8102 - val_loss: 0.4582 - val_acc: 0.7851
Epoch 2/10
40000/40000 [==============================] - 1087s 27ms/step - loss: 0.2724 - acc: 0.8934 - val_loss: 0.3058 - val_acc: 0.8835
Epoch 3/10
40000/40000 [==============================] - 1060s 26ms/step - loss: 0.1899 - acc: 0.9285 - val_loss: 0.3218 - val_acc: 0.8807
Epoch 4/10
40000/40000 [==============================] - 1056s 26ms/step - loss: 0.1373 - acc: 0.9497 - val_loss: 0.3343 - val_acc: 0.8841
Epoch 5/10
40000/40000 [==============================] - 1071s 27ms/step - loss: 0.1008 - acc: 0.9646 - val_loss: 0.3733 - val_acc: 0.8859
Epoch 6/10
4000

In [13]:
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 128, input_length=200))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(256, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(Bidirectional(LSTM(500, dropout=0.2, recurrent_dropout=0.2)))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])

model_conv.fit(data, df['label'], validation_split=0.2, epochs=10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 514s 13ms/step - loss: 0.3344 - acc: 0.8492 - val_loss: 0.2637 - val_acc: 0.8872
Epoch 2/10
40000/40000 [==============================] - 508s 13ms/step - loss: 0.1799 - acc: 0.9318 - val_loss: 0.2829 - val_acc: 0.8901
Epoch 3/10
40000/40000 [==============================] - 508s 13ms/step - loss: 0.1039 - acc: 0.9631 - val_loss: 0.3028 - val_acc: 0.8877
Epoch 4/10
40000/40000 [==============================] - 509s 13ms/step - loss: 0.0558 - acc: 0.9820 - val_loss: 0.3846 - val_acc: 0.8781
Epoch 5/10
40000/40000 [==============================] - 510s 13ms/step - loss: 0.0339 - acc: 0.9895 - val_loss: 0.5249 - val_acc: 0.8804
Epoch 6/10
40000/40000 [==============================] - 505s 13ms/step - loss: 0.0272 - acc: 0.9914 - val_loss: 0.5782 - val_acc: 0.8762
Epoch 7/10
40000/40000 [==============================] - 506s 13ms/step - loss: 0.0174 - acc: 0.9945 - val_loss: 0.